## Prepare and read data

In [16]:
import gc
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import lightgbm as lgb

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error

In [2]:
pd.set_option("max_columns", 300)
pd.set_option("max_rows", 500)

In [3]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

## Feature Engineering

In [4]:
def get_multi_cat_cols(train_df):
    tmp = train_df.iloc[:1000]
    multi_cols = []
    for c in train_df.columns:
        sep_num = tmp[c].astype(str).fillna("").str.contains(";").sum()
        if sep_num > 10:
            multi_cols.append(c)
    return multi_cols

In [5]:
original_cols = train_df.columns.tolist()

In [6]:
multi_cat_cols = get_multi_cat_cols(train_df)

In [7]:
for c in tqdm(multi_cat_cols):
    binarizer = MultiLabelBinarizer()
    train_multi_srs = train_df[c].map(lambda x: x.split(";") if x is not np.nan else [])
    test_multi_srs = test_df[c].map(lambda x: x.split(";") if x is not np.nan else [])
    train_arr = binarizer.fit_transform(train_multi_srs)
    test_arr = binarizer.transform(test_multi_srs)
    feat_cols = [f"ohe_{c}_{val}" for val in binarizer.classes_]
    train_feat_df = pd.DataFrame(train_arr, columns=feat_cols)
    test_feat_df = pd.DataFrame(test_arr, columns=feat_cols)
    train_feat_df[f"sum_answer_{c}"] = train_df[c].str.count(";") + 1
    test_feat_df[f"sum_answer_{c}"] = test_df[c].str.count(";") + 1
    train_df = pd.concat([train_df, train_feat_df], axis=1)
    test_df = pd.concat([test_df, test_feat_df], axis=1)

In [8]:
other_cols = [
    c for c in list(np.setdiff1d(original_cols, multi_cat_cols)) if c not in ["Salary", "No"]
]

In [9]:
cat_cols = []
for c in tqdm(other_cols):
    if "float" in train_df[c].dtype.name:
        continue
    else:
        train_df[c], uniques = pd.factorize(train_df[c], sort=True)
        test_df[c] = uniques.get_indexer(test_df[c])
        cat_cols.append(c)

In [10]:
train_df.shape, test_df.shape

((33857, 465), (11259, 464))

In [11]:
train_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in train_df.columns]
test_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in test_df.columns]

In [12]:
# basicなモデルのimportanceを読み込み
importance_df = pd.read_csv("../data/importance/003_importance.csv")

In [27]:
imp_feats = importance_df["feature"].iloc[:30].tolist()
imp_cat_cols = [c for c in imp_feats if c in cat_cols or c[:4] == "ohe_"]
imp_nume_cols = [c for c in imp_feats if c not in cat_cols and c not in multi_cat_cols]

In [28]:
len(imp_cat_cols), len(imp_nume_cols)

(21, 8)

In [29]:
all_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)
for cat_col in tqdm(imp_cat_cols):
    for nume_col in imp_nume_cols:
        print(nume_col)
        all_df[f"agg_mean_{cat_col}_{nume_col}"] = all_df.groupby(cat_col)[nume_col].transform("mean")
        all_df[f"agg_std_{cat_col}_{nume_col}"] = all_df.groupby(cat_col)[nume_col].transform("std")
        all_df[f"agg_max_{cat_col}_{nume_col}"] = all_df.groupby(cat_col)[nume_col].transform("max")
        all_df[f"agg_min_{cat_col}_{nume_col}"] = all_df.groupby(cat_col)[nume_col].transform("min")
        all_df[f"diff_agg_mean_{cat_col}_{nume_col}"] = all_df[nume_col] - all_df[f"agg_mean_{cat_col}_{nume_col}"]
        all_df[f"rel_agg_mean_{cat_col}_{nume_col}"] = all_df[nume_col] / (1 + all_df[f"agg_mean_{cat_col}_{nume_col}"])
train_df = all_df.iloc[:len(train_df)].reset_index(drop=True)
test_df = all_df.iloc[len(train_df):].reset_index(drop=True)
del all_df
gc.collect()

ohe_DevType_Student
ohe_RaceEthnicity_White_or_of_European_descent
AssessBenefits2
ohe_CommunicationTools_Confluence
ohe_DevType_Engineering_manager
ohe_LanguageWorkedWith_PHP
ohe_Methodology_Agile
ohe_EducationTypes_Contributed_to_open_source_software
ohe_DevType_Student
ohe_RaceEthnicity_White_or_of_European_descent
AssessBenefits2
ohe_CommunicationTools_Confluence
ohe_DevType_Engineering_manager
ohe_LanguageWorkedWith_PHP
ohe_Methodology_Agile
ohe_EducationTypes_Contributed_to_open_source_software
ohe_DevType_Student
ohe_RaceEthnicity_White_or_of_European_descent
AssessBenefits2
ohe_CommunicationTools_Confluence
ohe_DevType_Engineering_manager
ohe_LanguageWorkedWith_PHP
ohe_Methodology_Agile
ohe_EducationTypes_Contributed_to_open_source_software
ohe_DevType_Student
ohe_RaceEthnicity_White_or_of_European_descent
AssessBenefits2
ohe_CommunicationTools_Confluence
ohe_DevType_Engineering_manager
ohe_LanguageWorkedWith_PHP
ohe_Methodology_Agile
ohe_EducationTypes_Contributed_to_open_sour

480

In [30]:
use_cols = [c for c in train_df.columns if c not in multi_cat_cols + ["Salary", "No"]]

In [31]:
folds = KFold(n_splits=5, random_state=2020, shuffle=True)
fold_idx_list = [(trn_idx, val_idx) for trn_idx, val_idx in folds.split(train_df, train_df["Salary"])]

### Target Encoding

In [35]:
for c in tqdm(use_cols):
    if "agg_" in c or train_df[c].nunique() < 5:
        continue
    new_col = "te_" + c
    train_df[new_col] = 0
    test_df[new_col] = 0
    for trn_idx, val_idx in fold_idx_list:
        mean_val = train_df.loc[trn_idx].groupby(c)["Salary"].mean()
        train_df.loc[val_idx, new_col] = train_df.loc[val_idx, c].map(mean_val)
        test_df.loc[:, new_col] += test_df.loc[:, c].map(mean_val) / len(fold_idx_list)

In [38]:
use_cols = [c for c in train_df.columns if c not in multi_cat_cols + ["Salary", "No"]]
print(len(use_cols))

1556


## Training

In [39]:
lgb_params = {
            'objective': 'regression',
            "metric": "rmse",
            "verbosity": -1,
            "boosting": "gbdt",
            'learning_rate': 0.05,
            'num_leaves': 128,
            'min_data_in_leaf': 30, 
            'max_depth': 6,
            "bagging_freq": 10,
            "bagging_fraction": 0.4,
            "lambda_l1": 0.5,
            "lambda_l2": 0.5,
            "feature_fraction": 0.7,
            "seed": 2020,
            "num_threads": -1,
            "max_bins": 30
}

In [40]:
importances = pd.DataFrame()
oof_preds = np.zeros(len(train_df))
models = []

for fold_i, (trn_idx, val_idx) in enumerate(fold_idx_list):
    print(f"Fold {fold_i+1}")
    train_dataset = lgb.Dataset(
        train_df.loc[trn_idx, use_cols],
        label = train_df.loc[trn_idx, "Salary"]
    )
    valid_dataset = lgb.Dataset(
        train_df.loc[val_idx, use_cols],
        label = train_df.loc[val_idx, "Salary"]
    )
    model = lgb.train(
                lgb_params,
                train_dataset,
                3000,
                valid_sets = [train_dataset, valid_dataset],
                verbose_eval=100,
                early_stopping_rounds = 100,
                #feval = eval_f1,
                #callbacks = [log_callback],
    )
    imp_df = pd.DataFrame()
    imp_df['feature'] = use_cols
    imp_df['gain'] = model.feature_importance(importance_type="gain")
    importances = pd.concat([importances, imp_df], axis=0, sort=False)
    
    oof_preds[val_idx] = model.predict(train_df.loc[val_idx, use_cols])
    models.append(model)

Fold 1
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 19289.1	valid_1's rmse: 21493.6
[200]	training's rmse: 17759.8	valid_1's rmse: 21200.7
[300]	training's rmse: 16601.7	valid_1's rmse: 21106.6
[400]	training's rmse: 15751.5	valid_1's rmse: 21065
[500]	training's rmse: 14940.3	valid_1's rmse: 21117.5
Early stopping, best iteration is:
[402]	training's rmse: 15736.5	valid_1's rmse: 21063.3
Fold 2
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 19313.8	valid_1's rmse: 21418.5
[200]	training's rmse: 17610.7	valid_1's rmse: 20920.8
[300]	training's rmse: 16468.3	valid_1's rmse: 20830.3
[400]	training's rmse: 15525.3	valid_1's rmse: 20740.4
[500]	training's rmse: 14787.8	valid_1's rmse: 20759.3
Early stopping, best iteration is:
[440]	training's rmse: 15215.4	valid_1's rmse: 20732.9
Fold 3
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 19311.6	valid_1's rmse: 21347.1
[200]	trai

In [41]:
oof_score = np.sqrt(mean_squared_error(train_df["Salary"], oof_preds))
oof_score

20800.954809260507

In [42]:
importances.groupby("feature")["gain"].mean().sort_values(ascending=False).reset_index().iloc[:100]

,feature,gain
0,te_Country,7.463041e+13
1,agg_mean_YearsCodingProf_ohe_DevType_Student,2.182520e+13
2,agg_mean_Country_ohe_EducationTypes_Contribute...,6.832318e+12
3,te_CurrencySymbol,4.886347e+12
4,agg_std_YearsCodingProf_ohe_DevType_Student,4.008213e+12
5,agg_std_Country_ohe_EducationTypes_Contributed...,3.289845e+12
6,te_YearsCodingProf,2.903509e+12
7,agg_mean_Employment_ohe_DevType_Student,2.167784e+12
8,YearsCodingProf,1.816234e+12
9,diff_agg_mean_CompanySize_ohe_DevType_Engineer...,1.639602e+12


In [43]:
test_pred = np.zeros(len(test_df))

for model in models:
    test_pred += model.predict(test_df[use_cols]) / len(models)

In [44]:
test_pred

array([ 67207.6183046 ,  93006.37543944,  84744.84449588, ...,
        93401.61918869,  85782.80900668, 112611.09323446])

In [45]:
sub_df = pd.read_csv("../input/submit.csv")

In [46]:
sub_df["Salary"] = test_pred

In [47]:
sub_df.to_csv("../predict/007_baseline_te_20800.csv", index=False)